In [2]:
import numpy as np
import pandas as pd

In [5]:
class MyDecisionTree:
    def __init__(self, df):
        self.df = df
        self.prediction = None
        self.childs = None
        self.root = None
        self.labelsOfRoot = None
    def __fit__(self):
        if self.df.columns.size != 1:
            if len(np.unique(self.df.iloc[:, -1].to_numpy)) == 1:
                self.prediction = np.unique(self.df.iloc[:, -1].to_numpy)[0]
            else:
                self.root = self.find_best_feature()
                self.labelsOfRoot = np.unique(self.df[self.root].to_numpy())
                self.childs = np.empty((len(self.labelsOfRoot), ), dtype=object)
                for index, label in enumerate(self.labelsOfRoot):
                    self.childs[index] = MyDecisionTree(self.df[self.df[self.root] == label].drop(columns = self.root))
                    self.childs[index].__fit__()
        else:
            self.prediction = self.majority_vote()

    def majority_vote(self):
        labels = self.df.iloc[:, -1].to_numpy()
        return np.bincount(labels).argmax()
        

    def find_best_feature(self):
        features = self.df.columns.to_numpy()[:-1]
        enthropy = np.empty((len(features), ))
        for index, feature in enumerate(features):
            enthropy[index] = self.calc_entropy(feature)
        return features[np.argmin(enthropy)]    
    def calc_entropy(self, feature):
        labels = np.unique(self.df[feature].to_numpy())
        entropy = 0
        for label in labels:
            classDf = self.df[self.df[feature] == label]
            entropy += (len(classDf) / len(self.df)) * self.calc_class_entropy(classDf)
        return entropy    
    def calc_class_entropy(self, classDf):
        targets = classDf.iloc[:, -1].to_numpy()
        labels = np.unique(targets)
        perLabelEntropies = np.empty((len(labels), ))
        for index, label in enumerate(labels):
            labelCount = len(targets[targets == label])
            entropy = -(labelCount / len(classDf)) * (np.log(-((labelCount / len(classDf)))))
            perLabelEntropies[index] = entropy
        return np.sum(perLabelEntropies)    
    
    def __predict__(self, x):
        inputDict = {}
        for index, column in enumerate(self.df.columns.to_numpy()):
            inputDict[column] = x[index]
        if self.prediction == None:
            inputDict = {}
            for index, column in enumerate(self.df.columns.to_numpy()):
                inputDict[column] = x[index]
            return self.childs[np.where(self.labelsOfRoot == inputDict[self.root])].__predict__(x)
        else:
            return self.prediction    
            